# NLP Pipeline with spaCy

[spaCy](https://spacy.io/) is a widely used python library with a comprehensive feature set for fast text processing in multiple languages. 

The usage of the tokenization and annotation engines requires the installation of language models. The features we will use in this chapter only require the small models, the larger models also include word vectors that we will cover in chapter 15.

![spaCy](assets/spacy.jpg)

## Setup

### Imports

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
from talib import RSI

In [8]:
import sys
print(sys.executable)


/opt/anaconda3/envs/ml4t/bin/python


In [15]:
%matplotlib inline
import sys
from pathlib import Path

import pandas as pd

import spacy
from spacy import displacy
from textacy.extract import ngrams, entities

### SpaCy Language Model Installation

In addition to the `spaCy` library, we need [language models](https://spacy.io/usage/models).

#### English

Only need to run once.

In [16]:
%%bash
python -m spacy download en_core_web_sm

# more comprehensive models:
# {sys.executable} -m spacy download en_core_web_md
# {sys.executable} -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


#### Spanish

[Spanish language models](https://spacy.io/models/es#es_core_news_sm) trained on [AnCora Corpus](http://clic.ub.edu/corpus/) and [WikiNER](http://schwa.org/projects/resources/wiki/Wikiner)

Only need to run once.

In [17]:
%%bash
python -m spacy download es_core_news_sm

# more comprehensive model:
# {sys.executable} -m spacy download es_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 10.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


Create shortcut names

In [19]:
%%bash
python -m spacy link en_core_web_sm en;
python -m spacy link es_core_news_sm es;

⚠ As of spaCy v3.0, model symlinks are not supported anymore. You can
load trained pipeline packages using their full names or from a directory
path.


⚠ As of spaCy v3.0, model symlinks are not supported anymore. You can
load trained pipeline packages using their full names or from a directory
path.


#### Validate Installation

In [20]:
# validate installation
!{sys.executable} -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.4) =================
ℹ spaCy installation:
/opt/anaconda3/envs/ml4t/lib/python3.8/site-packages/spacy

NAME              SPACY            VERSION                            
es_core_news_sm   >=3.7.0,<3.8.0   3.7.0   ✔
en_core_web_sm    >=3.7.2,<3.8.0   3.7.1   ✔



## Get Data

In [21]:
DATA_DIR = Path('..', 'data')

- [BBC Articles](http://mlg.ucd.ie/datasets/bbc.html), use raw text files ([download](http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip))
    - Data already included in [data](../data) directory, just unzip before first-time use.
- [TED2013](http://opus.nlpl.eu/TED2013.php), a parallel corpus of TED talk subtitles in 15 langugages (sample provided) in `results/TED` subfolder of this directory.

## SpaCy Pipeline & Architecture

### The Processing Pipeline

When you call a spaCy model on a text, spaCy 

1) tokenizes the text to produce a `Doc` object. 

2) passes the `Doc` object through the processing pipeline that may be customized, and for the default models consists of
- a tagger, 
- a parser and 
- an entity recognizer. 

Each pipeline component returns the processed Doc, which is then passed on to the next component.

![Architecture](assets/pipeline.svg)

### Key Data Structures

The central data structures in spaCy are the **Doc** and the **Vocab**. Text annotations are also designed to allow a single source of truth:

- The **`Doc`** object owns the sequence of tokens and all their annotations. `Span` and `Token` are views that point into it. It is constructed by the `Tokenizer`, and then modified in place by the components of the pipeline. 
- The **`Vocab`** object owns a set of look-up tables that make common information available across documents. 
- The **`Language`** object coordinates these components. It takes raw text and sends it through the pipeline, returning an annotated document. It also orchestrates training and serialization.

![Architecture](assets/spaCy-architecture.svg)

## SpaCy in Action

### Create & Explore the Language Object

Once installed and linked, we can instantiate a spaCy language model and then call it on a document. As a result, spaCy produces a Doc object that tokenizes the text and processes it according to configurable pipeline components that by default consist of a tagger, a parser, and a named-entity recognizer.

In [44]:
nlp = spacy.load('en_core_web_sm') 

In [45]:
type(nlp)

spacy.lang.en.English

In [46]:
nlp.lang

'en'

In [55]:
spacy.info('en')

AttributeError: 'tuple' object has no attribute 'tb_frame'

In [56]:
def get_attributes(f):
    print([a for a in dir(f) if not a.startswith('_')], end=' ')

In [57]:
get_attributes(nlp)

['Defaults', 'add_pipe', 'analyze_pipes', 'batch_size', 'begin_training', 'component', 'component_names', 'components', 'config', 'create_optimizer', 'create_pipe', 'create_pipe_from_source', 'default_config', 'default_error_handler', 'disable_pipe', 'disable_pipes', 'disabled', 'enable_pipe', 'evaluate', 'factories', 'factory', 'factory_names', 'from_bytes', 'from_config', 'from_disk', 'get_factory_meta', 'get_factory_name', 'get_pipe', 'get_pipe_config', 'get_pipe_meta', 'has_factory', 'has_pipe', 'initialize', 'lang', 'make_doc', 'max_length', 'meta', 'path', 'pipe', 'pipe_factories', 'pipe_labels', 'pipe_names', 'pipeline', 'rehearse', 'remove_pipe', 'rename_pipe', 'replace_listeners', 'replace_pipe', 'resume_training', 'select_pipes', 'set_error_handler', 'set_factory_meta', 'to_bytes', 'to_disk', 'tokenizer', 'update', 'use_params', 'vocab'] 

### Explore the Pipeline

Let’s illustrate the pipeline using a simple sentence:

In [58]:
sample_text = 'Apple is looking at buying U.K. startup for $1 billion'
doc = nlp(sample_text)

In [59]:
get_attributes(doc)

['cats', 'char_span', 'copy', 'count_by', 'doc', 'ents', 'extend_tensor', 'from_array', 'from_bytes', 'from_dict', 'from_disk', 'from_docs', 'from_json', 'get_extension', 'get_lca_matrix', 'has_annotation', 'has_extension', 'has_unknown_spaces', 'has_vector', 'is_nered', 'is_parsed', 'is_sentenced', 'is_tagged', 'lang', 'lang_', 'mem', 'noun_chunks', 'noun_chunks_iterator', 'remove_extension', 'retokenize', 'sentiment', 'sents', 'set_ents', 'set_extension', 'similarity', 'spans', 'tensor', 'text', 'text_with_ws', 'to_array', 'to_bytes', 'to_dict', 'to_disk', 'to_json', 'to_utf8_array', 'user_data', 'user_hooks', 'user_span_hooks', 'user_token_hooks', 'vector', 'vector_norm', 'vocab'] 

In [60]:
doc.is_parsed

True

In [61]:
doc.is_sentenced

True

In [62]:
doc.is_tagged

True

In [63]:
doc.text

'Apple is looking at buying U.K. startup for $1 billion'

In [64]:
get_attributes(doc.vocab)

['add_flag', 'cfg', 'deduplicate_vectors', 'from_bytes', 'from_disk', 'get_noun_chunks', 'get_vector', 'has_vector', 'lang', 'length', 'lex_attr_getters', 'lookups', 'morphology', 'prune_vectors', 'reset_vectors', 'set_vector', 'strings', 'to_bytes', 'to_disk', 'vectors', 'vectors_length', 'writing_system'] 

In [65]:
doc.vocab.length

773

#### Explore `Token` annotations

The parsed document content is iterable and each element has numerous attributes produced by the processing pipeline. The below sample illustrates how to access the following attributes:

In [66]:
pd.Series([token.text for token in doc])

0       Apple
1          is
2     looking
3          at
4      buying
5        U.K.
6     startup
7         for
8           $
9           1
10    billion
dtype: object

In [67]:
pd.DataFrame([[t.text, t.lemma_, t.pos_, t.tag_, t.dep_, t.shape_, t.is_alpha, t.is_stop]
              for t in doc],
             columns=['text', 'lemma', 'pos', 'tag', 'dep', 'shape', 'is_alpha', 'is_stop'])

,text,lemma,pos,tag,dep,shape,is_alpha,is_stop
0,Apple,Apple,PROPN,NNP,nsubj,Xxxxx,True,False
1,is,be,AUX,VBZ,aux,xx,True,True
2,looking,look,VERB,VBG,ROOT,xxxx,True,False
3,at,at,ADP,IN,prep,xx,True,True
4,buying,buy,VERB,VBG,pcomp,xxxx,True,False
5,U.K.,U.K.,PROPN,NNP,dobj,X.X.,False,False
6,startup,startup,NOUN,NN,dep,xxxx,True,False
7,for,for,ADP,IN,prep,xxx,True,True
8,$,$,SYM,$,quantmod,$,False,False
9,1,1,NUM,CD,compound,d,False,False


#### Visualize POS Dependencies

We can visualize the syntactic dependency in a browser or notebook

In [68]:
options = {'compact': True, 'bg': 'white',
           'color': 'black', 'font': 'Source Sans Pro', 'notebook': True}

In [69]:
displacy.render(doc, style='dep', options=options)

#### Visualize Named Entities

In [92]:
displacy.render(doc, style='ent', jupyter=True)

### Read BBC Data

We will now read a larger set of 2,225 BBC News articles (see GitHub for data source details) that belong to five categories and are stored in individual text files. We 
- call the .glob() method of the pathlib’s Path object, 
- iterate over the resulting list of paths, 
- read all lines of the news article excluding the heading in the first line, and 
- append the cleaned result to a list

In [93]:
files = (DATA_DIR / 'bbc').glob('**/*.txt')
bbc_articles = []
for i, file in enumerate(sorted(list(files))):
    with file.open(encoding='latin1') as f:
        lines = f.readlines()
        body = ' '.join([l.strip() for l in lines[1:]]).strip()
        bbc_articles.append(body)

In [94]:
len(bbc_articles)

2225

In [95]:
bbc_articles[0]

'Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.  Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL\'s underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL\'s existing customers for high-

### Parse first article through Pipeline

In [96]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [97]:
doc = nlp(bbc_articles[0])
type(doc)

spacy.tokens.doc.Doc

### Detect sentence boundary
Sentence boundaries are calculated from the syntactic parse tree, so features such as punctuation and capitalisation play an important but non-decisive role in determining the sentence boundaries. 

Usually this means that the sentence boundaries will at least coincide with clause boundaries, even given poorly punctuated text.

spaCy computes sentence boundaries from the syntactic parse tree so that punctuation and capitalization play an important but not decisive role. As a result, boundaries will coincide with clause boundaries, even for poorly punctuated text.

We can access the parsed sentences using the .sents attribute:

In [98]:
sentences = [s for s in doc.sents]
sentences[:3]

[Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.  ,
 The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales.,
 TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.]

In [99]:
get_attributes(sentences[0])

['as_doc', 'char_span', 'conjuncts', 'doc', 'end', 'end_char', 'ent_id', 'ent_id_', 'ents', 'get_extension', 'get_lca_matrix', 'has_extension', 'has_vector', 'id', 'id_', 'kb_id', 'kb_id_', 'label', 'label_', 'lefts', 'lemma_', 'n_lefts', 'n_rights', 'noun_chunks', 'orth_', 'remove_extension', 'rights', 'root', 'sent', 'sentiment', 'sents', 'set_extension', 'similarity', 'start', 'start_char', 'subtree', 'tensor', 'text', 'text_with_ws', 'to_array', 'vector', 'vector_norm', 'vocab'] 

In [100]:
pd.DataFrame([[t.text, t.pos_, spacy.explain(t.pos_)] for t in sentences[0]], 
             columns=['Token', 'POS Tag', 'Meaning']).head(15)

,Token,POS Tag,Meaning
0,Quarterly,ADJ,adjective
1,profits,NOUN,noun
2,at,ADP,adposition
3,US,PROPN,proper noun
4,media,NOUN,noun
5,giant,NOUN,noun
6,TimeWarner,PROPN,proper noun
7,jumped,VERB,verb
8,76,NUM,numeral
9,%,NOUN,noun


In [101]:
options = {'compact': True, 'bg': '#09a3d5',
           'color': 'white', 'font': 'Source Sans Pro'}
displacy.render(sentences[0].as_doc(), style='dep', jupyter=True, options=options)

In [102]:
for t in sentences[0]:
    if t.ent_type_:
        print('{} | {} | {}'.format(t.text, t.ent_type_, spacy.explain(t.ent_type_)))

Quarterly | DATE | Absolute or relative dates or periods
US | GPE | Countries, cities, states
TimeWarner | ORG | Companies, agencies, institutions, etc.
76 | PERCENT | Percentage, including "%"
% | PERCENT | Percentage, including "%"
1.13bn | MONEY | Monetary values, including unit
Â£600 | CARDINAL | Numerals that do not fall under another type
the | DATE | Absolute or relative dates or periods
three | DATE | Absolute or relative dates or periods
months | DATE | Absolute or relative dates or periods
to | DATE | Absolute or relative dates or periods
December | DATE | Absolute or relative dates or periods
639 | MONEY | Monetary values, including unit
year | DATE | Absolute or relative dates or periods
- | DATE | Absolute or relative dates or periods
earlier | DATE | Absolute or relative dates or periods


In [103]:
displacy.render(sentences[0].as_doc(), style='ent', jupyter=True)

### Named Entity-Recognition with textacy

spaCy enables named entity recognition using the .ent_type_ attribute:

Textacy makes access to the named entities that appear in the first article easy:

In [120]:
from textacy.extract import ngrams, entities

entities = [e.text for e in entities(doc)]
pd.Series(entities).value_counts().head()

TimeWarner        7
AOL               5
fourth quarter    3
AOL Europe        2
SEC               2
Name: count, dtype: int64

### N-Grams with textacy

N-grams combine N consecutive tokens. This can be useful for the bag-of-words model because, depending on the textual context, treating, e.g, ‘data scientist’ as a single token may be more meaningful than the two distinct tokens ‘data’ and ‘scientist’.

Textacy makes it easy to view the ngrams of a given length n occurring with at least min_freq times:

In [121]:
pd.Series([n.text for n in ngrams(doc, n=2, min_freq=2)]).value_counts()

fourth quarter     3
Time Warner        2
quarter profits    2
company said       2
AOL Europe         2
Name: count, dtype: int64

### The spaCy streaming Pipeline API

To pass a larger number of documents through the processing pipeline, we can use spaCy’s streaming API as follows:

In [129]:
iter_texts = (bbc_articles[i] for i in range(len(bbc_articles)))
for i, doc in enumerate(nlp.pipe(iter_texts, batch_size=50, n_process=8)):
    if i % 100 == 0:
        print(i, end = ' ')
    assert doc.is_parsed

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 

### Multi-language Features

spaCy includes trained language models for English, German, Spanish, Portuguese, French, Italian and Dutch, as well as a multi-language model for named-entity recognition. Cross-language usage is straightforward since the API does not change.

We will illustrate the Spanish language model using a parallel corpus of TED talk subtitles. For this purpose, we instantiate both language models

#### Create a Spanish Language Object

In [123]:
model = {}
for language in ['en', 'es']:
    model[language] = spacy.load(language) 

OSError: [E941] Can't find model 'en'. It looks like you're trying to load a model from a shortcut, which is obsolete as of spaCy v3.0. To load the model, use its full name instead:

nlp = spacy.load("en_core_web_sm")

For more details on the available models, see the models directory: https://spacy.io/models and if you want to create a blank model, use spacy.blank: nlp = spacy.blank("en")

#### Read bilingual TED2013 samples

In [ ]:
text = {}
path = Path('data', 'TED')
for language in ['en', 'es']:
    file_name = path /  'TED2013_sample.{}'.format(language)
    text[language] = file_name.read_text()

#### Sentence Boundaries English vs Spanish

In [124]:
parsed, sentences = {}, {}
for language in ['en', 'es']:
    parsed[language] = model[language](text[language])
    sentences[language] = list(parsed[language].sents)
    print('Sentences:', language, len(sentences[language]))

KeyError: 'en'

In [125]:
for i, (en, es) in enumerate(zip(sentences['en'], sentences['es']), 1):
    print('\n', i)
    print('English:\t', en)
    print('Spanish:\t', es)
    if i > 5: 
        break

KeyError: 'en'

#### POS Tagging English vs Spanish

In [126]:
pos = {}
for language in ['en', 'es']:
    pos[language] = pd.DataFrame([[t.text, t.pos_, spacy.explain(t.pos_)] for t in sentences[language][0]],
                                 columns=['Token', 'POS Tag', 'Meaning'])

KeyError: 'en'

In [127]:
bilingual_parsed = pd.concat([pos['en'], pos['es']], axis=1)
bilingual_parsed.head(15)

KeyError: 'en'

In [128]:
displacy.render(sentences['es'][0].as_doc(), style='dep', jupyter=True, options=options)

KeyError: 'es'